In [ ]:
%store -r WEAVIATE_IP
from boto3 import Session

session = Session()
credentials = session.get_credentials()
current_credentials = credentials.get_frozen_credentials()

AWS_ACCESS_KEY = current_credentials.access_key
AWS_SECRET_KEY = current_credentials.secret_key
AWS_SECRET_TOKEN = current_credentials.token
print(f"AWS_ACCESS_KEY:\t{AWS_ACCESS_KEY}")
print(f"AWS_SECRET_KEY:\t{AWS_SECRET_KEY}")
print(f"AWS_SECRET_TOKEN:\t{AWS_SECRET_TOKEN}")
print(f"WEAVIATE_IP:\t{WEAVIATE_IP}")

## Connect

In [ ]:
import weaviate

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_IP, http_port="8080",  http_secure=False,
    grpc_host=WEAVIATE_IP, grpc_port="50051", grpc_secure=False,

    headers={
        "X-AWS-Access-Key": AWS_ACCESS_KEY,
        "X-AWS-Secret-Key": AWS_SECRET_KEY,
        "X-AWS-Session-Token": AWS_SECRET_TOKEN,
    }
)

client.is_ready()

## Search
### On `title` vector

In [ ]:
import weaviate.classes.query as wq

# Get the collection
movies = client.collections.get("MoviesMM")

# Perform query
response = movies.query.near_text(
    query="Movie about lovable cute pets",
    # query="Epic super hero",
    target_vector="content",  # The target vector to search against
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_url"]
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

### Get the BLOB object for the poster
By default, the Weaviate doesn't return blob objects, as they tend to be large, and it is a lot more efficient to skip them.

In order to get the `poster`, you need to use `return_properties` and request it along other required properties.

In [ ]:
import weaviate.classes.query as wq

# Get the collection
movies = client.collections.get("MoviesMM")

# Perform query
response = movies.query.near_text(
    query="Movie about lovable cute pets",
    # query="Epic super hero",
    target_vector="content",  # The content target vector
    limit=5,
    return_properties=["title", "release_date", "tmdb_id", "poster_path", "poster"], # return poster blob
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_path"],
    )

    # Print the poster blob
    print(o.properties["poster"])

    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

### Image search – On `poster` vector

In [ ]:
import weaviate.classes.query as wq

# Get the collection
movies = client.collections.get("MoviesMM")

# Perform query
response = movies.query.near_text(
    query="Epic super hero",
    target_vector="poster",  # The poster target vector
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_path"],
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

### `Image search` on `poster` vector

In [ ]:
from IPython.display import Image
display(Image("./test/mouse.jpg", width=150))
display(Image("./test/droid.jpeg", width=150))
display(Image("./test/spooky.jpg", width=250))

In [ ]:
import weaviate.classes.query as wq
from pathlib import Path

# Get the collection
movies = client.collections.get("MoviesMM")

# Perform query
response = movies.query.near_image(
    # near_image=Path("./test/mouse.jpg"),
    # near_image=Path("./test/droid.jpeg"),
    near_image=Path("./test/spooky.jpg"),
    target_vector="poster",  # The target vector to search against
    limit=5,
    return_metadata=wq.MetadataQuery(distance=True),
)

# Inspect the response
for o in response.objects:
    display(Image(o.properties["poster_path"], width=200))

    # Print the title and release year (note the release date is a datetime object)
    print(
        o.properties["title"],
        o.properties["release_date"].year,
        o.properties["tmdb_id"],
        o.properties["poster_path"]
    )

    # Print the distance of the object from the query
    print(
        f"Distance to query: {o.metadata.distance:.3f}\n"
    )

In [ ]:
client.close()